# Try reading gnomAD/PanUKBB data into memory

In [2]:
using Revise
using EasyLD

┌ Info: Precompiling EasyLD [a5d00db5-5f0a-49b9-aaff-6f4121a13ee1]
└ @ Base loading.jl:1423


In [3]:
datadir = "/Users/biona001/.julia/dev/EasyLD/data"
bm_file = joinpath(datadir, "UKBB.EUR.ldadj.bm")
ht_file = joinpath(datadir, "UKBB.EUR.ldadj.variant.ht")
bm = hail_block_matrix(bm_file, ht_file);

Size of matrix

In [4]:
size(bm)

2023-03-14 11:39:53.534 WARN  Utils:69 - Your hostname, Benjamins-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 10.0.0.86 instead (on interface en0)
2023-03-14 11:39:53.538 WARN  Utils:69 - Set SPARK_LOCAL_IP if you need to bind to another address
2023-03-14 11:39:54.131 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Initializing Hail with default parameters...
Picked up _JAVA_OPTIONS: -Xmx16g
Picked up _JAVA_OPTIONS: -Xmx16g
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://10.0.0.86:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.109-b71b065e4bb6
LOGGING: writing to /Users/biona001/.julia/dev/EasyLD/test/hail-20230314-1139-0.2.109-b71b065e4bb6.log


(23960350, 23960350)

Read a single entry

In [5]:
bm[1, 1]

0.999959841549239

Read a block

In [6]:
@time bm[1:1000, 1:1000]

  2.739247 seconds (236.01 k allocations: 20.288 MiB, 3.86% compilation time)


1000×1000 Matrix{Float64}:
 0.99996  0.00098559  -0.000830543  …   0.00427643    0.000288337
 0.0      0.999942    -0.00104453      -0.000365344  -0.0014734
 0.0      0.0          0.999963        -0.000545369  -0.00101798
 0.0      0.0          0.0             -0.00052585   -0.00124045
 0.0      0.0          0.0              0.00195088    7.25693e-5
 0.0      0.0          0.0          …  -0.000118419  -0.000835871
 0.0      0.0          0.0             -9.25639e-5    0.00120466
 0.0      0.0          0.0             -0.000214606   0.000833805
 0.0      0.0          0.0             -0.000274162  -0.000390875
 0.0      0.0          0.0             -0.000110892   0.00294388
 0.0      0.0          0.0          …  -0.000373815  -0.000942022
 0.0      0.0          0.0             -0.000288357  -0.00193506
 0.0      0.0          0.0             -0.000162489  -0.00150939
 ⋮                                  ⋱                
 0.0      0.0          0.0             -0.000357731  -0.00113869
 0.0 

In [19]:
# indexing non-contiguous region is slow
@time bm[1:3, 1:2:100]

 22.868453 seconds (61.51 k allocations: 3.108 MiB, 0.31% compilation time)


3×50 Matrix{Float64}:
 1.0  -0.0181891   0.00802797  -0.0421512  …  0.0143534     0.0172358
 0.0  -0.00041394  0.00232164  -0.0112066     5.39059e-5   -0.0188801
 0.0   1.0         0.0         -0.0282011     0.000733417  -0.0277042

Read a specific chromosome region

In [24]:
chr = 1 # string or Int
start_pos = 11063
end_pos = 91588
sigma = get_block(bm, chr, start_pos, end_pos)

9×9 Matrix{Float64}:
 0.99996  0.00098559  -0.000830543  …  -0.00045888    0.000221516
 0.0      0.999942    -0.00104453      -0.000332668  -0.000439517
 0.0      0.0          0.999963        -0.000551892  -0.000786793
 0.0      0.0          0.0             -0.000503539  -0.000712834
 0.0      0.0          0.0             -0.000294435   0.00575015
 0.0      0.0          0.0          …  -0.000112519  -0.00015419
 0.0      0.0          0.0             -0.000264108   0.00413586
 0.0      0.0          0.0              0.999978     -0.000225057
 0.0      0.0          0.0              0.0           0.99995

## Testing

In [44]:
using PyCall
py"""
from hail.linalg import BlockMatrix
import numpy as np
def convert(bm, range_start, range_end):
    return bm[range_start:range_end, range_start:range_end].to_numpy()
"""
@time py"convert"(bm.bm, 0, 100)

  0.827913 seconds (33.87 k allocations: 1.972 MiB, 1.93% compilation time)


100×100 Matrix{Float64}:
 1.0  0.0326361  -0.0181891   0.0166341   …   0.0172358    -0.00857776
 0.0  1.0        -0.00041394  0.00540477     -0.0188801     0.05925
 0.0  0.0         1.0         0.0177243      -0.0277042    -0.000268512
 0.0  0.0         0.0         1.0            -0.000468342  -0.000966708
 0.0  0.0         0.0         0.0            -7.47553e-5   -0.00298802
 0.0  0.0         0.0         0.0         …   0.00207735    0.00152017
 0.0  0.0         0.0         0.0             0.00256425    0.00136302
 0.0  0.0         0.0         0.0             0.0137416    -0.000680087
 0.0  0.0         0.0         0.0            -0.020671      0.000342287
 0.0  0.0         0.0         0.0             0.00195264   -0.00451867
 0.0  0.0         0.0         0.0         …   0.0018285    -0.00406988
 0.0  0.0         0.0         0.0             0.0152337     0.0100539
 0.0  0.0         0.0         0.0             0.0193       -0.0136828
 ⋮                                        ⋱          

### Python code

In [ ]:
from hail.linalg import BlockMatrix
# import numpy as np
bm_files = "UKBB.EUR.ldadj.bm"
bm = BlockMatrix.read(bm_files)

# get a slice fast
x = bm[0:10000, 0:10000]
X = x.to_numpy()
X

In [ ]:
def convert(bm, range1, range2)
    return bm[range1, range2].to_numpy()